In [1]:
# LangGraph Hotel Finder Chatbot (Thailand)
# ตามสไตล์จาก 04.ipynb: ใช้ StateGraph + MemorySaver + Gradio

import json
import os
import time
from typing import Annotated, Dict, Any

from dotenv import load_dotenv
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
import gradio as gr

load_dotenv(override=True)



True

In [2]:
# State และ helper รวมค่าแบบ 04.ipynb

def add_value(left, right):
    if right is not None:
        return right
    return left

class State(BaseModel):
    user_input: Annotated[str, add_value] = ""
    location: Annotated[str, add_value] = ""
    start_date: Annotated[str, add_value] = ""
    end_date: Annotated[str, add_value] = ""
    additional_info: Annotated[str, add_value] = ""
    search_results: Annotated[str, add_value] = ""
    hotels_markdown: Annotated[str, add_value] = ""
    reply_text: Annotated[str, add_value] = ""
    timestamp: Annotated[float, add_value] = 0.0



In [3]:
# เตรียม LLM + Tool + Graph

checkpointer = MemorySaver()
graph_builder = StateGraph(State)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

serper = GoogleSerperAPIWrapper()
tool_search = Tool(
    name="hotel_search",
    func=serper.run,
    description="ค้นหาข้อมูลโรงแรมตาม location และช่วงวันที่"
)



In [4]:
# Node 1: แยก intent เป็น location/start/end/additional ด้วย GPT-4o

def parse_request(state: State):
    def strip_code_fence(text: str) -> str:
        t = text.strip()
        if t.startswith("```"):
            lines = t.splitlines()
            if lines and lines[0].startswith("```"):
                lines = lines[1:]
            if lines and lines[-1].strip() == "```":
                lines = lines[:-1]
            t = "\n".join(lines).strip()
        return t

    prompt = (
        "ให้ช่วยดึงข้อมูลการจองโรงแรมจากข้อความผู้ใช้ เป็น JSON key: location, start_date, end_date, additional_info (optional). "
        "ถ้าผู้ใช้ระบุเฉพาะ start_date และจำนวนคืน/จำนวนวัน (เช่น 3 วัน) ให้คำนวณ end_date ให้ครบ. "
        "ถ้าระบุ start_date อย่างเดียว ไม่ระบุจำนวนวัน ให้สมมติพัก 1 คืน และกำหนด end_date เป็นวันถัดไป. "
        "ถ้าไม่พบข้อมูลให้ใส่ค่าว่าง. รูปแบบวันที่ให้คงข้อความเดิมหรือคำนวณเป็นข้อความที่เข้าใจง่ายในภาษาไทย. ตอบกลับเฉพาะ JSON.\n"
        f"ข้อความผู้ใช้: {state.user_input}"
    )
    resp = llm.invoke(prompt)
    raw = strip_code_fence(str(resp.content))
    try:
        data = json.loads(raw)
    except Exception:
        data = {}
    return {
        "location": data.get("location", ""),
        "start_date": data.get("start_date", ""),
        "end_date": data.get("end_date", ""),
        "additional_info": data.get("additional_info", state.additional_info),
        "reply_text": ""
    }



In [5]:
# Node 2: ตรวจว่าข้อมูลครบหรือยัง ถ้าไม่ครบให้ถามกลับ

def check_required(state: State):
    missing = []
    if not state.location:
        missing.append("สถานที่ (location)")
    if not state.start_date:
        missing.append("วันที่เริ่มเข้าพัก (start_date)")
    if not state.end_date:
        missing.append("วันที่สิ้นสุด (end_date)")
    if missing:
        ask = "รบกวนแจ้ง " + ", ".join(missing) + " เพื่อค้นหาโรงแรมครับ/ค่ะ"
        return {"reply_text": ask}
    return {"reply_text": ""}



In [6]:
# Node 3: เรียก Serper หาโรงแรมตามเงื่อนไข

def fetch_hotels(state: State):
    query_parts = [
        f"โรงแรม {state.location}",
        f"วันที่ {state.start_date} ถึง {state.end_date}"
    ]
    if state.additional_info:
        query_parts.append(state.additional_info)
    query = " ".join(query_parts)
    result = tool_search.run(query)
    return {"search_results": result}



In [7]:
# Node 4: LLM สรุปผลเป็น Markdown แสดงรายชื่อ ราคา และเหตุผลที่ตรง requirement

def build_recommendation(state: State):
    def strip_code_fence(text: str) -> str:
        t = text.strip()
        if t.startswith("```"):
            lines = t.splitlines()
            if lines and lines[0].startswith("```"):
                lines = lines[1:]
            if lines and lines[-1].strip() == "```":
                lines = lines[:-1]
            t = "\n".join(lines).strip()
        return t

    prompt = (
        "คุณคือผู้ช่วยท่องเที่ยว ช่วยสรุปผลการค้นหาโรงแรมจากข้อมูลด้านล่างเป็น Markdown bullet list. "
        "แต่ละรายการควรมีชื่อโรงแรม, ช่วงราคา/ราคาต่อคืน (ถ้าพบ), และเหตุผลที่ตรงกับ requirement เช่น วิวทะเล, pet friendly, งบไม่เกิน. "
        "ให้ตอบเป็นภาษาไทย กระชับ.\n"
        f"Location: {state.location}\nStartDate: {state.start_date}\nEndDate: {state.end_date}\nAdditionalInfo: {state.additional_info}\n"
        f"ผลค้นหา (raw): {state.search_results}"
    )
    resp = llm.invoke(prompt)
    md = strip_code_fence(str(resp.content))
    ts = time.time()
    return {"hotels_markdown": md, "reply_text": md, "timestamp": ts}



In [8]:
# Node 5: ส่งข้อความตอบกลับ

def output_reply(state: State):
    return state



In [9]:
# Wire graph

graph_builder.add_node("parse_request", parse_request)
graph_builder.add_node("check_required", check_required)
graph_builder.add_node("fetch_hotels", fetch_hotels)
graph_builder.add_node("build_recommendation", build_recommendation)
graph_builder.add_node("output_reply", output_reply)

graph_builder.add_edge(START, "parse_request")
graph_builder.add_edge("parse_request", "check_required")

def next_after_check(state: State):
    # ถ้ามีข้อความ reply_text แสดงว่าขาดข้อมูล -> ส่งออกเพื่อถามผู้ใช้
    if state.reply_text:
        return "output_reply"
    return "fetch_hotels"


graph_builder.add_conditional_edges("check_required", next_after_check)
graph_builder.add_edge("fetch_hotels", "build_recommendation")
graph_builder.add_edge("build_recommendation", "output_reply")
graph_builder.add_edge("output_reply", END)

graph = graph_builder.compile(checkpointer=checkpointer)



In [10]:
# ฟังก์ชันหลักสำหรับ Gradio Chat

def chat_handler(message, history: list[tuple[str, str]]):
    initial_state = State(user_input=message)
    result_state = graph.invoke(
        initial_state,
        config={"configurable": {"thread_id": "hotel-chat"}}
    )
    return result_state["reply_text"]



In [11]:
# สร้าง Gradio ChatInterface

chat_ui = gr.ChatInterface(
    fn=chat_handler,
    title="Hotel Finder Chatbot",
    description="พิมพ์สถานที่ วันที่เข้า-ออก และความต้องการเพิ่มเติม เพื่อค้นหาโรงแรม",
)

if __name__ == "__main__":
    chat_ui.launch()



d:\Demo\agents\LangGraphPractice\.venv\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
